# Otimização Stardew Valley

Neste notebook, o objetivo é trabalhar com algoritmos de otimização diversos, desde os clássicos até os famosos bio-inspirados. Mas antes de pensar nos algoritmos, vamos pensar no problema.

Podemos pensar em casos de otimização com muita facilidade, desde aqueles casos clássicos da literatura de roteamento entre cidades (e o famoso problema do caixeiro viajante, TSP) até os processos de decisão do nosso dia a dia, como comprar a maior diversidade de produtos no mercado com um valor fixo de dinheiro disponível. Em outras palavras, estudos de caso não faltam para a gente implementar métodos de otimização.

Dito isto, não vamos testar problemas clássicos demais aqui, como o TSP. Por outro lado, vamos otimizar as plantações de um jogo chamado Stardew Valley, um simulador de fazenda bem popular. O objetivo é, para uma determina estação e uma disponibilidade de espaços para plantio, qual a melhor configuração de cultivos maximiza o lucro. As plantas disponíveis podem ser encontradas na [wiki](https://pt.stardewvalleywiki.com/Lavouras)

In [95]:
import math
import numpy as np
from enum import Enum
from typing import List, Tuple
import matplotlib.pyplot as plt
from typing_extensions import Self
from IPython.display import clear_output

In [133]:
class Season(Enum):
    PRIMAVERA = 1
    VERAO = 2
    OUTONO = 3
    INVERNO = 4

class Plant:
    def __init__(self, name: str, season: Season, buy_price: int, days_yield: int,
        quantity_yield: int, sell_price: int, days_after_yield: int=None
    ) -> None:
        self.name = name
        self.season = season
        self.buy_price = buy_price
        self.days_yield = days_yield
        self.quantity_yield = quantity_yield
        self.sell_price = sell_price
        self.days_after_yield = days_after_yield

    def get_profit(self, time_period: int, log=False) -> int:
        buy = self.buy_price
        growth = 0
        sell = 0
        loop = 0

        def show(title: str, day: int) -> None:
            print(f"Ação de {title} executada no dia {day}")

        for i in range(1, time_period + 1):
            # Vende uma plantação, se ela não for perene
            if growth == self.days_yield and self.days_after_yield is None:
                sell = sell + self.quantity_yield * self.sell_price
                if log: show("venda", i)
                loop = loop + 1
                growth = 0

            # Vende uma plantação perene
            if self.days_after_yield is not None and growth == self.days_after_yield:
                sell = sell + self.quantity_yield * self.sell_price
                if log: show("venda", i)
                loop = loop + 1
                growth = 0 

            # Se não for possível mais colheitas, finaliza o loop
            if self.days_after_yield is None and (time_period - i) + growth < self.days_yield:
                break

            if self.days_after_yield is not None and (time_period - i) + growth < self.days_after_yield:
                break

            # No dia zero, depois de uma colheita, faz a replantação, se
            # a cultura não for perene
            if growth == 0 and loop > 0 and self.days_after_yield is None:
                buy = buy + self.buy_price
                if log: show("compra", i)
            
            growth = growth + 1

        return sell - buy

class Crop:
    def __init__(self, plant: Plant, quantity: int) -> None:
        self.plant = plant
        self.quantity = quantity
    
    def set_quantity(self, quantity: int) -> Self:
        self.quantity = quantity
        return self

class StardewFarm:
    def __init__(self, max_spaces: int) -> None:
        self.max_spaces = max_spaces
        self.market = {
            Season.PRIMAVERA.name: [
                Plant("Alho", Season.PRIMAVERA, 40, 4, 1, 60, days_after_yield=None),
                Plant("Arroz", Season.PRIMAVERA, 40, 8, 1, 30, days_after_yield=None),
                Plant("Batata", Season.PRIMAVERA, 50, 6, 1, 80, days_after_yield=None),
                Plant("Chirívia", Season.PRIMAVERA, 20, 4, 1, 35, days_after_yield=None),
                Plant("Couve", Season.PRIMAVERA, 70, 6, 1, 110, days_after_yield=None),
                Plant("Couve-flor", Season.PRIMAVERA, 80, 12, 1, 175, days_after_yield=None),
                Plant("Café", Season.PRIMAVERA, 2500, 10, 4, 15, days_after_yield=2),
                Plant("Jasmim Azul", Season.PRIMAVERA, 30, 7, 1, 50, days_after_yield=None),
                Plant("Morango", Season.PRIMAVERA, 100, 8, 1, 120, days_after_yield=4),
                Plant("Ruibarbo", Season.PRIMAVERA, 100, 13, 1, 220, days_after_yield=None),
                Plant("Tulipa", Season.PRIMAVERA, 20, 6, 1, 30, days_after_yield=None),
                Plant("Feijão", Season.PRIMAVERA, 60, 10, 1, 40, days_after_yield=3)
            ],
            Season.VERAO.name: [],
            Season.OUTONO.name: [],
            Season.INVERNO: []
        }

    def random_plants(self, season: Season) -> List[Crop]:
        max_range = self.max_spaces // 5
        solution: List[Crop] = []
        def total(plan: List[Crop]) -> int:
            return sum([c.quantity for c in plan])

        for plant in self.market[season.name]:
            quantity = np.random.randint(0, max_range)
            current_quantity = total(solution)
            if quantity + current_quantity > self.max_spaces:
                quantity = self.max_spaces - current_quantity
            solution.append(Crop(plant, quantity))

        return solution

    def get_neighbors(self, crops: List[int], number: int) -> List[List[int]]:
        neighbors: List[List[int]] = []
        while len(neighbors) < number:
            i = np.random.randint(0, len(crops))
            j = np.random.randint(0, len(crops))
            if i == j:
                continue
            if crops[i] == 0:
                continue

            remove = int(np.random.random_sample() * crops[i])
            if remove == 0:
                continue
            
            neighbor = crops.copy()
            neighbor[i] = neighbor[i] - remove
            neighbor[j] = neighbor[j] + remove

            if sum(neighbor) > self.max_spaces:
                continue

            neighbors.append(neighbor)
        
        return neighbors
    
    def get_profit(self, season: Season, time_period: int, solution: List[int]) -> int:
        total: int = 0
        for i in range(0, len(solution)):
            total = total + self.market[season.name][i].get_profit(time_period, log=False) * solution[i]
        return total
    
    def crossover(self, p1: List[int], p2: List[int]) -> Tuple[List[int], List[int]]:
        def populate(child: List[int], parent: List[int]) -> None:
            for j in range(len(child), len(parent)):
                add = parent[j]
                current = sum(child)
                if current + add > self.max_spaces:
                    add = self.max_spaces - current
                child.append(add)

        i = np.random.randint(1, len(p1) - 1)
        c1, c2 = p1[:i], p2[:i]
        populate(c1, p2)
        populate(c2, p1)
        return c1, c2
    
    def mutate(self, solution: List[int]) -> None:
        if np.random.random_sample() < 0.2:
            i = np.random.randint(0, len(solution))
            j = np.random.randint(0, len(solution))
            while i == j:
                j = np.random.randint(0, len(solution))

            solution[i], solution[j] = solution[j], solution[i]
    
    def tournament(self, season: Season, time_period: int, population: List[List[int]]) -> List[int]:
        i = np.random.randint(0, len(population))
        j = np.random.randint(0, len(population))

        ifitness = self.get_profit(season, time_period, population[i])
        jfitness = self.get_profit(season, time_period, population[j])

        if ifitness > jfitness:
            return population[i]
        else:
            return population[j]
        
    def best_individual(self, population: List[List[int]], season: Season, time_period: int) -> List[int]:
        best_solution, best_fitness = None, 0
        for solution in population:
            profit = self.get_profit(season, time_period, solution)
            if profit > best_fitness:
                best_fitness = profit
                best_solution = solution
        return best_solution
    
    def genetic_algorithm(self, season: Season, time_period: int, n_generations=1000, n_population=50):
        population = [[c.quantity for c in self.random_plants(season)] for _ in range(0, n_population)]
        best_hist: List[List[int]] = []

        for i in range(0, n_generations):
            new_population: List[List[int]] = []
            while len(new_population) != n_population:
                p1 = self.tournament(season, time_period, population)
                p2 = self.tournament(season, time_period, population)
                c1, c2 = self.crossover(p1, p2)
                self.mutate(c1)
                self.mutate(c2)
                new_population.append(c1)
                new_population.append(c2)
            
            best_solution = self.best_individual(new_population, season, time_period)
            profit = self.get_profit(season, time_period, best_solution)
            best_hist.append(best_solution)
            population = new_population

            print(f"Geração {i + 1}, fitness: R$ {profit}", end="\r")
            #print(f"Melhor fitness: R$ {profit}")
        
        return best_solution

    def show_plan(self, season: Season, solution: List[int]) -> None:
        print(f"Planejamento de cultivo: ocupação de {sum(solution)} espaços")
        for i in range(0, len(solution)):
            print(f"{solution[i]} de {self.market[season.name][i].name}")

In [61]:
garlic = Plant("Alho", Season.PRIMAVERA, 40, 4, 1, 60, 2)
print(garlic.get_profit(13, log=False))

320


In [134]:
farm = StardewFarm(200)
solution = farm.genetic_algorithm(Season.PRIMAVERA, 30, n_generations=1000, n_population=50)
profit = farm.get_profit(Season.PRIMAVERA, 30, solution)
print()
print(f"Lucro otimizado: R$ {profit}")
farm.show_plan(Season.PRIMAVERA, solution)

Geração 1000, fitness: R$ 60450
Lucro otimizado: R$ 60450
Planejamento de cultivo: ocupação de 200 espaços
20 de Alho
0 de Arroz
5 de Batata
0 de Chirívia
35 de Couve
35 de Couve-flor
0 de Café
0 de Jasmim Azul
35 de Morango
35 de Ruibarbo
0 de Tulipa
35 de Feijão


In [110]:
for product in farm.market[Season.PRIMAVERA.name]:
    profit = product.get_profit(30) * 200
    print(f"Monocultura de {product.name}: R$ {profit}")

Monocultura de Alho: R$ 28000
Monocultura de Arroz: R$ -6000
Monocultura de Batata: R$ 24000
Monocultura de Chirívia: R$ 21000
Monocultura de Couve: R$ 32000
Monocultura de Couve-flor: R$ 38000
Monocultura de Café: R$ -332000
Monocultura de Jasmim Azul: R$ 16000
Monocultura de Morango: R$ 148000
Monocultura de Ruibarbo: R$ 48000
Monocultura de Tulipa: R$ 8000
Monocultura de Feijão: R$ 60000


In [93]:
farm = StardewFarm(200)
plan = farm.random_plants(Season.PRIMAVERA)
initial_solution = [c.quantity for c in plan]
neighbors = farm.get_neighbors(initial_solution, 2)
profit = farm.get_profit(Season.PRIMAVERA, 28, initial_solution)

print(f"Solução inicial {sum(initial_solution)} (R$ {profit}): {initial_solution}")
for neighbor in neighbors:
    profit = farm.get_profit(Season.PRIMAVERA, 28, neighbor)
    print(f"Solução vizinha {sum(neighbor)} (R$ {profit}): {neighbor}")

c1, c2 = farm.crossover(neighbors[0], neighbors[1])
print()
print(f"Filho 1 {sum(c1)} (R$ {farm.get_profit(Season.PRIMAVERA, 28, c1)}): {c1}")
print(f"Filho 2 {sum(c2)} (R$ {farm.get_profit(Season.PRIMAVERA, 28, c2)}): {c2}")

Solução inicial 159 (R$ -13420): [13, 14, 24, 20, 4, 12, 21, 17, 12, 14, 1, 7]
Solução vizinha 159 (R$ -7820): [13, 14, 24, 20, 4, 12, 21, 7, 22, 14, 1, 7]
Solução vizinha 159 (R$ 12340): [27, 14, 24, 20, 4, 12, 7, 17, 12, 14, 1, 7]

Filho 1 159 (R$ -13420): [13, 14, 24, 20, 4, 12, 21, 17, 12, 14, 1, 7]
Filho 2 159 (R$ 17940): [27, 14, 24, 20, 4, 12, 7, 7, 22, 14, 1, 7]
